### Model evaluation

We evaluat ehte three models using different 

1. Pseudo FID: We compute the Frecht distance between the CLIP () embdeddings of the dataset and .... generated images.
2. LPIP sdisnrtace

We first load the necessary code.

In [1]:
import torchvision
import torch
import json
import os
import sys
import importlib
import shutil
import numpy as np 
import lightning as L
import tqdm

from lightning.pytorch.utilities.model_summary import ModelSummary
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
from torchvision.transforms import v2

plt.rcParams["savefig.bbox"] = 'tight'

# Adding the package in loading path
sys.path.extend(["../"])

import gencellpainting as gc
from gencellpainting.utils.dataset import WGANCriticDataset, CellPaintingDatasetInMemory
from gencellpainting.model import *
from gencellpainting.evaluation.clip_fih import FrechetCLIPDistance

/home/adelabriere/miniconda3/envs/imaging/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Path of the .pt fil
PATH_DATASET = "/mnt/c/Users/alexi/Documents/data/images/cellpainting/cpg0016-jump/data/jump_64px_uint8.pt"
# Path of saved models
PATH_OUTPUT= os.path.join(os.path.abspath("../.."),"output")
PATH_MODEL = os.path.join(PATH_OUTPUT,"models")


In [ ]:
NREPEAT = 3

We first load the  full dataset, which will be used to compute the distribution of CLIP embeddings

In [5]:
ds = CellPaintingDatasetInMemory(tensor=torch.load(PATH_DATASET))
ds = torch.utils.data.Subset(ds,range(5000))
dl = torch.utils.data.DataLoader(ds, batch_size=64)

We load the pretrained models

In [6]:
wgangp = WGAN_GP.load_from_checkpoint(os.path.join(PATH_MODEL,"WGANGP")).to("cuda")
diffusion = DiffusionProcess.load_from_checkpoint(os.path.join(PATH_MODEL,"DIFFUNET")).to("cuda")
vae = VAE.load_from_checkpoint(os.path.join(PATH_MODEL,"VAE")).to("cuda")
MODELS = {
    "VAE":vae,
    "WGANGP":wgangp,
    "Diff-UNet":diffusion
}

/home/adelabriere/miniconda3/envs/imaging/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


We define a function to update the FID.

In [7]:
def compute_fih(dl, model):
    with torch.no_grad():
        clip_distance = FrechetCLIPDistance().to("cuda")
        total_samples = 0
        for batch in tqdm.tqdm(dl):
            B, _, _, _ = batch.size()
            total_samples += B
            batch = batch.float().to("cuda")
            clip_distance.update(batch, is_real=True)
            fake_images = model.generate_images(batch=batch,n = B)
            fake_images = fake_images.to("cuda")
            clip_distance.update(fake_images, is_real=False)
        return clip_distance.compute(), total_samples
    

In [ ]:
fihs = {}
for key,model in MODELS.items():
    metrics = []
    for i in range(3):
        met,_ = compute_fih(dl, model)
        metrics.append(float(met))
    fihs[key] = metrics

  0%|          | 0/79 [00:00<?, ?it/s]/home/adelabriere/miniconda3/envs/imaging/lib/python3.12/site-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv2d(
100%|██████████| 79/79 [50:36<00:00, 38.44s/it]


In [9]:
MODELS.keys()

dict_keys(['VAE', 'WGANGP', 'Diff-UNet'])

In [11]:
with open(os.path.join(PATH_OUTPUT,"fihs.json"),"w") as f:
    json.dump(fihs,f)


In [ ]:
records = {(key,float(np.mean(values)),float(np.std(values))) for key,values in fihs.items()}

In [ ]:
VAE.load_from_checkpoint(os.path.join(PATH_MODEL,"VAE"))
